In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.layers import Conv2D
from time import time
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [3]:
df1 = pd.read_hdf('df_dataDOL1.hdf', key='hdf')
df2 = pd.read_hdf('df_dataDOL2.hdf', key='hdf')
df3 = pd.read_hdf('df_dataDOL3.hdf', key='hdf')

In [4]:
df_total = pd.concat([df1,df2,df3],  ignore_index=True)

In [6]:
df_total.head()

,Data,Contratos Abertos,Contratos Fechados,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,Var pontos,Última oferta de compra,Última oferta de venda
0,29/07/2019,851537,719017,27192,226625,42939771125,3.7850,3.7775,3.8015,3.789499,3.7820,3.784797,145520,3.7820,3.7825
1,26/07/2019,875047,851537,32128,309480,58414828000,3.7800,3.7610,3.7955,3.775030,3.7760,3.770245,125890,3.7765,3.7780
2,25/07/2019,890941,875047,41651,381060,72105906125,3.7680,3.7570,3.8070,3.784490,3.7795,3.782834,129780,3.7795,3.7805
3,24/07/2019,883412,890941,26137,269620,50725252625,3.7675,3.7540,3.7795,3.762721,3.7735,3.769856,31160,3.7735,3.7755
4,23/07/2019,880504,883412,33586,313840,59121712875,3.7450,3.7430,3.7820,3.767634,3.7750,3.772972,356000,3.7750,3.7770


In [7]:
df = df_total.copy()

df['Dif_contratos'] = df['Contratos Abertos'] -df['Contratos Fechados']
df['Dif_minmax'] = (df['Máximo'] - df['Mínimo'])*1000
df['Dif_abert_ultimo'] = (df['Último Preço'] - df['Abertura'])*1000
df['Dif_abert_ajuste'] = (df['Ajuste'] - df['Abertura'])*1000
df['Dif_abert_min'] = (df['Abertura'] - df['Mínimo'])*1000
df['Dif_abert_max'] = (df['Abertura'] - df['Máximo'])*1000
df['Dif_abert_medio'] = (df['Abertura'] - df['Médio'])*1000
df['Dif_ajuste_ultimo'] = (df['Ajuste'] - df['Último Preço'])*1000
df['Dif_ajuste_medio'] = (df['Ajuste'] - df['Médio'])*1000
df['Dif_ultimacomp_ultimavend'] = (df['Última oferta de venda'] - df['Última oferta de compra'])*1000

df['Contratos Negociados'] = df['Contratos Negociados']/10000
df['Número Negócios'] = df['Número Negócios']/100
df['Volume'] = df['Volume']/1000000000

df = df.drop(columns = ['Contratos Abertos', 'Contratos Fechados'])
df.head()

,Data,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,...,Dif_contratos,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend
0,29/07/2019,271.92,22.6625,42.939771,3.7850,3.7775,3.8015,3.789499,3.7820,3.784797,...,132520,24.0,-3.0,-0.203,7.5,-16.5,-4.499,2.797,-4.702,0.5
1,26/07/2019,321.28,30.9480,58.414828,3.7800,3.7610,3.7955,3.775030,3.7760,3.770245,...,23510,34.5,-4.0,-9.755,19.0,-15.5,4.970,-5.755,-4.785,1.5
2,25/07/2019,416.51,38.1060,72.105906,3.7680,3.7570,3.8070,3.784490,3.7795,3.782834,...,15894,50.0,11.5,14.834,11.0,-39.0,-16.490,3.334,-1.656,1.0
3,24/07/2019,261.37,26.9620,50.725253,3.7675,3.7540,3.7795,3.762721,3.7735,3.769856,...,-7529,25.5,6.0,2.356,13.5,-12.0,4.779,-3.644,7.135,2.0
4,23/07/2019,335.86,31.3840,59.121713,3.7450,3.7430,3.7820,3.767634,3.7750,3.772972,...,-2908,39.0,30.0,27.972,2.0,-37.0,-22.634,-2.028,5.338,2.0


In [22]:
df.reindex(index=df.index[::-1]).reset_index(drop = True)

,Data,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,...,Dif_contratos,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend
0,04/09/1997,9.05,25.6900,14.134870,1.10080,1.09990,1.10085,1.100418,1.10005,1.100084,...,-55533,0.95,-0.75,-0.716,0.90,-0.05,0.382,0.034,-0.334,0.0
1,05/09/1997,6.64,18.2110,10.013101,1.09980,1.09940,1.09990,1.099676,1.09955,1.099478,...,46865,0.50,-0.25,-0.322,0.40,-0.10,0.124,-0.072,-0.198,0.0
2,08/09/1997,5.73,14.2050,7.807353,1.09930,1.09900,1.09970,1.099240,1.09960,1.099599,...,23656,0.70,0.30,0.299,0.30,-0.40,0.060,-0.001,0.359,0.0
3,09/09/1997,7.71,20.1055,11.052047,1.09950,1.09915,1.09965,1.099405,1.09940,1.099425,...,36925,0.50,-0.10,-0.075,0.35,-0.15,0.095,0.025,0.020,0.0
4,10/09/1997,5.28,12.9865,7.138234,1.09920,1.09900,1.09970,1.099332,1.09965,1.099658,...,18820,0.70,0.45,0.458,0.20,-0.50,-0.132,0.008,0.326,0.0
5,11/09/1997,8.13,20.0879,11.046988,1.09970,1.09965,1.10010,1.099865,1.09980,1.099718,...,-11074,0.45,0.10,0.018,0.05,-0.40,-0.165,-0.082,-0.147,0.0
6,12/09/1997,6.10,15.9440,8.765348,1.09960,1.09930,1.09970,1.099517,1.09940,1.099510,...,5280,0.40,-0.20,-0.090,0.30,-0.10,0.083,0.110,-0.007,0.0
7,15/09/1997,6.14,13.8040,7.588279,1.09930,1.09920,1.09960,1.099432,1.09940,1.099458,...,-13225,0.40,0.10,0.158,0.10,-0.30,-0.132,0.058,0.026,0.0
8,16/09/1997,5.51,16.6120,9.129514,1.09950,1.09875,1.09960,1.099147,1.09885,1.098828,...,18418,0.85,-0.65,-0.672,0.75,-0.10,0.353,-0.022,-0.319,0.0
9,17/09/1997,4.60,13.0545,7.171014,1.09875,1.09840,1.09880,1.098627,1.09860,1.098591,...,16370,0.40,-0.15,-0.159,0.35,-0.05,0.123,-0.009,-0.036,0.0
